# Item-item collaborative filtering
RecSys Challenge 2022 - Group 5

## General Data Preparation

Let us read in the different datasets.

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
base_path_train = "~/shared/data/project/training"

items_df = pd.read_csv(os.path.join(base_path_train, "item_features.csv"))
purchase_df = pd.read_csv(os.path.join(base_path_train, "train_purchases.csv"))
session_df = pd.read_csv(os.path.join(base_path_train, "train_sessions.csv"))

In [3]:
items_df

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75
...,...,...,...
471746,28143,68,351
471747,28143,55,390
471748,28143,11,109
471749,28143,73,91


In [4]:
items_df.item_id.nunique()

23691

In [5]:
purchase_df

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114
...,...,...,...
999995,4439986,2915,2021-05-13 11:56:37.464
999996,4439990,8786,2020-08-22 14:28:22.382
999997,4439994,21630,2020-11-27 20:10:28.961
999998,4439999,16962,2020-11-27 11:01:41.356


In [6]:
session_df

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211
...,...,...,...
4743815,4440001,20409,2020-10-30 23:37:20.658
4743816,4440001,14155,2020-10-30 23:31:56.607
4743817,4440001,14303,2020-10-30 23:36:17.934
4743818,4440001,27852,2020-10-30 23:39:55.186


Now we combine the views inside a session and the purchases of this session in one dataframe, with the column `was_bought` indicating whether the item was only viewed or bought.

In [7]:
purchase_df_processed = purchase_df.copy()
purchase_df_processed["was_bought"] = True

session_df_processed = session_df.copy()
session_df_processed["was_bought"] = False
df_processed = pd.concat([purchase_df_processed, session_df_processed]).sort_values(["session_id", "date"])
df_processed

,session_id,item_id,date,was_bought
1,3,9655,2020-12-18 21:19:48.093,False
0,3,9655,2020-12-18 21:25:00.373,False
0,3,15085,2020-12-18 21:26:47.986,True
2,13,15654,2020-03-13 19:35:27.136,False
1,13,18626,2020-03-13 19:36:15.507,True
...,...,...,...,...
4743804,4440001,19539,2020-10-30 23:37:09.46,False
4743815,4440001,20409,2020-10-30 23:37:20.658,False
4743818,4440001,27852,2020-10-30 23:39:55.186,False
4743806,4440001,20449,2020-10-30 23:40:28.149,False


Now we denormalize the item features table, to have a more handy representation of the item features

In [8]:
items_processed_df = items_df.pivot_table(values='feature_value_id', index='item_id', columns='feature_category_id').reset_index()
items_processed_df.index.names = ['index']
items_processed_df.columns = ["item_id"] + [f"item_feature_{x}" for x in list(range(73))]
items_processed_df

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
index,,,,,,,,,,,,,,,,,,,,,
0,2,NaN,NaN,NaN,NaN,NaN,NaN,394.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,NaN
1,3,NaN,NaN,889.0,618.0,605.0,NaN,452.0,NaN,NaN,...,NaN,521.0,NaN,NaN,14.0,592.0,NaN,NaN,75.0,544.0
2,4,NaN,NaN,793.0,618.0,605.0,NaN,837.0,NaN,NaN,...,NaN,521.0,NaN,NaN,373.0,538.0,NaN,NaN,75.0,544.0
3,7,NaN,NaN,NaN,NaN,NaN,NaN,536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,739.0,592.0,NaN,NaN,75.0,NaN
4,8,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,...,NaN,521.0,NaN,NaN,351.0,592.0,NaN,NaN,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,...,NaN,521.0,NaN,NaN,106.0,805.0,NaN,NaN,75.0,544.0
23687,28140,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.0,NaN,NaN,349.0,351.0,NaN,NaN,226.0,NaN,544.0
23688,28141,461.0,NaN,889.0,719.0,605.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,379.0,499.0,NaN,NaN,75.0,544.0


The item features can now be merged to the combined dataset with session views and purchases from above.
Also NULL values are filled by 0.

In [9]:
df_processed = df_processed.merge(items_processed_df, how="left", on="item_id")
df_processed["was_bought"] = df_processed["was_bought"].astype(float)
df_processed

,session_id,item_id,date,was_bought,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
0,3,9655,2020-12-18 21:19:48.093,0.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,393.0,NaN,NaN,NaN,NaN,544.0
1,3,9655,2020-12-18 21:25:00.373,0.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,393.0,NaN,NaN,NaN,NaN,544.0
2,3,15085,2020-12-18 21:26:47.986,1.0,NaN,53.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,349.0,97.0,NaN,NaN,NaN,NaN,544.0
3,13,15654,2020-03-13 19:35:27.136,0.0,NaN,NaN,NaN,618.0,NaN,766.0,...,NaN,521.0,NaN,NaN,351.0,780.0,NaN,NaN,219.0,NaN
4,13,18626,2020-03-13 19:36:15.507,1.0,NaN,NaN,793.0,618.0,605.0,NaN,...,NaN,NaN,NaN,NaN,739.0,805.0,NaN,NaN,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743815,4440001,19539,2020-10-30 23:37:09.46,0.0,NaN,NaN,NaN,618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,544.0
5743816,4440001,20409,2020-10-30 23:37:20.658,0.0,NaN,NaN,NaN,618.0,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,544.0
5743817,4440001,27852,2020-10-30 23:39:55.186,0.0,NaN,NaN,NaN,618.0,NaN,778.0,...,NaN,550.0,NaN,NaN,351.0,362.0,NaN,NaN,75.0,544.0
5743818,4440001,20449,2020-10-30 23:40:28.149,0.0,NaN,NaN,NaN,618.0,NaN,778.0,...,NaN,550.0,NaN,NaN,351.0,362.0,NaN,NaN,75.0,544.0


In [10]:
items_processed_df = items_processed_df.fillna(0)
items_processed_df

,item_id,item_feature_0,item_feature_1,item_feature_2,item_feature_3,item_feature_4,item_feature_5,item_feature_6,item_feature_7,item_feature_8,...,item_feature_63,item_feature_64,item_feature_65,item_feature_66,item_feature_67,item_feature_68,item_feature_69,item_feature_70,item_feature_71,item_feature_72
index,,,,,,,,,,,,,,,,,,,,,
0,2,0.0,0.0,0.0,0.0,0.0,0.0,394.0,0.0,0.0,...,0.0,0.0,0.0,0.0,351.0,885.0,0.0,0.0,75.0,0.0
1,3,0.0,0.0,889.0,618.0,605.0,0.0,452.0,0.0,0.0,...,0.0,521.0,0.0,0.0,14.0,592.0,0.0,0.0,75.0,544.0
2,4,0.0,0.0,793.0,618.0,605.0,0.0,837.0,0.0,0.0,...,0.0,521.0,0.0,0.0,373.0,538.0,0.0,0.0,75.0,544.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,536.0,0.0,0.0,...,0.0,0.0,0.0,0.0,739.0,592.0,0.0,0.0,75.0,0.0
4,8,0.0,0.0,793.0,618.0,605.0,0.0,798.0,0.0,0.0,...,0.0,521.0,0.0,0.0,351.0,592.0,0.0,0.0,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,0.0,0.0,793.0,618.0,605.0,0.0,798.0,0.0,0.0,...,0.0,521.0,0.0,0.0,106.0,805.0,0.0,0.0,75.0,544.0
23687,28140,0.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,80.0,0.0,0.0,349.0,351.0,0.0,0.0,226.0,0.0,544.0
23688,28141,461.0,0.0,889.0,719.0,605.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,379.0,499.0,0.0,0.0,75.0,544.0


In [11]:
item_id2index = dict(zip(items_processed_df.item_id, items_processed_df.index))

In [12]:
all_items = list(items_processed_df["item_id"])

In [13]:
items_processed_array = np.array(items_processed_df.drop("item_id",axis=1))
items_processed_array[item_id2index[2]]

array([  0.,   0.,   0.,   0.,   0.,   0., 394.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,  38.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 123.,   0.,   0.,   0., 802.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0., 123.,   0.,   0.,  76.,   0.,   0.,   6.,   0.,   0.,
       365.,   0.,   0.,   0.,   0., 462., 801.,   0.,   0.,   0.,   0.,
         0., 351., 885.,   0.,   0.,  75.,   0.])

In [14]:
items_processed_array[item_id2index[2]].shape

(73,)

Next we read in the candidate items.

In [15]:
candidate_items = list(pd.read_csv("candidate_items.csv")["item_id"])
candidate_items[:10]

[4, 8, 9, 19, 20, 26, 33, 40, 51, 54]

## Item-item collaborative filtering specific data preparation

Similar to assignment 1 from the course, we will prepare our data for item-item collaborative filtering.

For this we also include the test data, in order to be able to do the predictions in the end for these test sessions.

In order to handle implicit feedback, we will set the rating of bought items to 1 and the rating of seen but not bought items to 0.5. The idea behind this is that seen items shall have a higher rating then non-seen items (which were not of interest in this session). This way, we capture how interesting an item was with respect to a session.

First we load necessary modules.

In [16]:
import os
import csv
import pandas as pd
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp

Now we read in the test data and set the rating of seen items to 0.5.

In [17]:
base_path_test = "~/shared/data/project/test"


test_df = pd.read_csv(os.path.join(base_path_test, "test_sessions.csv"))

In [18]:
test_df["rating"]=0.5
test_df = test_df.drop("date", axis=1)

In [19]:
test_sessions = test_df.session_id.unique()
len(test_sessions)

50000

For the training data we set the rating of seen (but not bought) items to 0.5, and the rating of purchased items to 1.

In [20]:
df_processed.loc[df_processed.was_bought==True, "rating"] = 1
df_processed.loc[df_processed.was_bought==False, "rating"] = 0.5
df_processed = pd.concat([df_processed, test_df])

In [21]:
ratings_raw = df_processed[["session_id", "item_id", "rating"]]
ratings_raw

,session_id,item_id,rating
0,3,9655,0.5
1,3,9655,0.5
2,3,15085,1.0
3,13,15654,0.5
4,13,18626,1.0
...,...,...,...
197619,186456690,10471,0.5
197620,186456690,13385,0.5
197621,186456690,10471,0.5
197622,186456690,5382,0.5


As in assignement 1, a ratings dataframe is built. To make life easier, we convert the session and item IDs to consecutive numbers.

In [22]:
itemIds = ratings_raw.item_id.unique()
itemIds.sort()
sessionIds = ratings_raw.session_id.unique()
sessionIds.sort()

## create internal ids for movies and users, that have consecutive indexes starting from 0
itemId_to_itemIDX = dict(zip(itemIds, range(0, itemIds.size)))
itemIDX_to_itemId = dict(zip(range(0, itemIds.size), itemIds))

sessionId_to_sessionIDX = dict(zip(sessionIds, range(0, sessionIds.size )))
sessionIDX_to_sessionId = dict(zip(range(0, sessionIds.size), sessionIds))

## drop timestamps
ratings = pd.concat([ratings_raw['session_id'].map(sessionId_to_sessionIDX), ratings_raw['item_id'].map(itemId_to_itemIDX), ratings_raw['rating']], axis=1)
ratings.columns = ['session', 'item', 'rating']

ratings.head()

,session,item,rating
0,0,8092,0.5
1,0,8092,0.5
2,0,12661,1.0
3,1,13153,0.5
4,1,15667,1.0


In [23]:
test_sessions_idx = [sessionId_to_sessionIDX[s] for s in test_sessions]
candidate_items_idx = [itemId_to_itemIDX[i] for i in candidate_items if i in itemId_to_itemIDX.keys()]

Now we build the session-item matrix `R`, and compute the item and session averages.

In [24]:
R = sp.csr_matrix((ratings.rating, (ratings.session, ratings.item)))
R_dok = R.todok()

m = R.shape[0]
n = R.shape[1]
numRatings = R.count_nonzero()

print("There are", m, "sessions,", n, "items and", numRatings, "ratings.")

There are 1049714 sessions, 23618 items and 5284342 ratings.


In [25]:
item_sums = R.sum(axis=0).A1 ## matrix converted to 1-D array via .A1
item_cnts = (R != 0).sum(axis=0).A1
item_avgs = item_sums / item_cnts
print("item_avgs", item_avgs)

item_avgs [0.5        0.57803468 0.6381733  ... 0.64480874 0.5        0.61428571]


In [26]:
session_sums = R.sum(axis=1).A1 ## matrix converted to 1-D array via .A1
session_cnts = (R != 0).sum(axis=1).A1
session_avgs = session_sums / session_cnts
print("session_avgs", session_avgs)

session_avgs [1.    0.75  0.625 ... 0.5   0.9   0.5  ]


## Computing item similarities

As indicated in lecture 4, slide 8, it makes sense to normalize user vectors when working with implicit data. Thus, we will do that in the next step.

In [27]:
# normalizing
norms = np.array(np.sqrt(R.T.multiply(R.T).sum(axis=0)))
norms[norms == 0.0] = 0.00001 # avoid dividing by 0
norms = norms.reshape((1,m))

nc = np.repeat(norms, session_cnts)
nc = nc.reshape(R.data.shape)

R.data /= nc

Similar to assignment 1, we write a function computing the item similarities for a given item ID. 

In [28]:
def compute_item_similarities(i_id):
    iI = np.empty((n,))

    # YOUR CODE HERE
    Rc = R.copy()
    
    # mean-centering
    #mc = np.repeat(item_avgs, item_cnts)
    #Rc.data -= mc
    
    # extracting user similarities
    i = Rc[:,i_id]
    iI = np.array(Rc.T.dot(i).todense())
    iI = iI.reshape((n,))
    
    return iI

In [29]:
iI = compute_item_similarities(121)
iI[10]

0.022727272727272728

To make our computations faster, we precompute item-similarities for the candidate items. The results are stored in a dictionary mapping the item ID to its item similarities.

In [30]:
iI_dict = {}

for i in candidate_items_idx:
    iI = compute_item_similarities(i)
    iI_dict[i] = iI

## Prediction of implicit feedback for a specific (user, item) pair

In the next step, we write a function which returns a rating prediction (prediction of implicit feedback). This is done according to lecture 10, slide 14: The "prediction for a target item is the sum of similarities of all current session items".

In [31]:
## a default value
with_deviations = True

def predict_rating(u_id, i_id, have_rated_idx):
    iI = iI_dict[i_id]
    
    # create new iI and fill it with 0 and then fill it with iI for items having been rated by the session (i.e. which were seen)
    iI_new = np.full(iI.shape, 0)
    iI_new[have_rated_idx] = iI[have_rated_idx]
    
    prediction = iI_new.sum()
    
    return prediction

In [32]:
import time

have_rated_idx = np.array([j_id for j_id in range(n) if (4, j_id) in R_dok])
start = time.time()
print(predict_rating(4, candidate_items_idx[1], have_rated_idx))
end = time.time()
end - start

0


0.0002796649932861328

## Prediction on test data

Now we will write a function, which for a given session and each candidate item computes the predicted rating according to the above function. This score will then be used for ranking our candidate items.

In [33]:
def predict_session(s):
    candidate_rank_dict = {}
    have_rated_idx = np.array([j_id for j_id in range(n) if (s, j_id) in R_dok])
    for i in candidate_items_idx:
        
        p = predict_rating(s, i, have_rated_idx)
        
        candidate_item = itemIDX_to_itemId[i]
        candidate_rank_dict[candidate_item] = p

    candidate_rank_df = pd.DataFrame(candidate_rank_dict.items(), columns = ["item_id", "score"])
    candidate_rank_df = candidate_rank_df.sort_values("score", ascending=False).head(100).reset_index(drop=True)
    candidate_rank_df["rank"] = candidate_rank_df.index + 1
    session_id = sessionIDX_to_sessionId[s]
    candidate_rank_df["session_id"] = session_id
    candidate_rank_df = candidate_rank_df.drop("score", axis=1)[["session_id", "item_id", "rank"]]
    return candidate_rank_df

This function is applied for all test sessions and the result is written to a csv file.

In [34]:
start = time.time()
candidate_rank_df_list = []
for s in test_sessions_idx:
    
    candidate_rank_df = predict_session(s)
    candidate_rank_df_list.append(candidate_rank_df)

candidate_rank_df = pd.concat(candidate_rank_df_list)
end = time.time()
print(end - start)

candidate_rank_df

6784.767990350723


,session_id,item_id,rank
0,126,4,1
1,126,18770,2
2,126,18809,3
3,126,18801,4
4,126,18798,5
...,...,...,...
95,186479748,17788,96
96,186479748,18953,97
97,186479748,17789,98
98,186479748,18968,99


In [35]:
candidate_rank_df.to_csv("results_iicf.csv", index=False)

In [36]:
# len(candidate_rank_df[candidate_rank_df.score == 0]) / len(candidate_rank_df)